# Amazon SageMaker MLOps: da ideia à produção em seis passos

Esta sequência de seis notebooks o guia desde o desenvolvimento de sua ideia de ML em um notebook simples até uma solução de produção com pipelines automatizados de construção de modelos e implantação de CI/CD, e monitoramento de modelos.

Siga estas etapas uma a uma:
1. Experimente em um notebook
2. Mova os scripts para o SageMaker SDK usando os Jobs de processamento e treinamento de dados 
3. Adicione um ML pipeline, um  model registry, um feature store 
4. Adicione um pipeline de CI/CD de construção de modelos
5. Adicionar um pipeline de CI/CD de implantação de modelo
6. Adicionar monitoramento de data quality

![](img/seis-etapas.png)

Existem também exemplos práticos adicionais de outros recursos do SageMaker e tópicos de ML, como testes A/B (https://docs.aws.amazon.com/sagemaker/latest/dg/model-validation.html), processamento personalizado (https://docs.aws.amazon.com/sagemaker/latest/dg/build-your-own-processing-container.html), contêineres de treinamento (https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-training-algo.html) e inferência (https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-inference-main.html), depuração e perfil (https://docs.aws.amazon.com/sagemaker/latest/dg/train-debugger.html), segurança (https://docs.aws.amazon.com/sagemaker/latest/dg/security.html), endpoints multi-modelo (https://docs.aws.amazon.com/sagemaker/latest/dg/multi-model-endpoints.html) e multi-contêiner (https://docs.aws.amazon.com/sagemaker/latest/dg/multi-container-endpoints.html), e pipelines de inferência serial (https://docs.aws.amazon.com/sagemaker/latest/dg/inference-pipelines.html). Explore os notebooks na pasta additional-topics para testar esses recursos.



Para executar este notebook e todos os notebooks do repositório, utilize o kernel Python 3 no JupyterLab.

## Favorite o Repositório no GitHub

In [1]:
%%html

<a class="github-button" href="https://github.com/CleidianePrates/amazon-sagemaker-from-idea-to-production.git" data-color-scheme="no-preference: light; light: light; dark: dark;" data-icon="octicon-star" data-size="large" data-show-count="true" aria-label="Star Amazon SageMaker secure MLOps on GitHub">Star</a>
<script async defer src="https://buttons.github.io/buttons.js"></script>

### Clique neste botão ^^^ acima ^^^

## Configuração
Obtenha a versão mais recente do SageMaker Python SDK.


<div class="alert alert-info"> 💡 Todos os notebooks desse repositório foram testados na versão 2.219.0 do SageMaker Python SDK (o pacote sagemaker). Os notebooks não fixam a versão do sagemaker. Se você encontrar algum problema de incompatibilidade, pode instalar a versão específica do sagemaker executando o comando pip:<code>%pip install sagemaker=2.219.0</code>
</div>

<svg width="800" height="125" viewBox="0 0 800 125" xmlns="http://www.w3.org/2000/svg">
    <defs>
        <linearGradient id="fadeGradient" x1="0" x2="1">
            <stop offset="0%" stop-color="#F0F0F0"/>
            <stop offset="100%" stop-color="#F0F0F0" stop-opacity="0"/>
        </linearGradient>
        <mask id="fadeMask">
            <rect x="0" y="0" width="750" height="125" fill="white"/>
            <rect x="750" y="0" width="50" height="125" fill="url(#fadeGradient)"/>
        </mask>
    </defs>
    <path d="M3,50 A50,50 0 0 1 53,3 L797,3 L797,97 L97,97 L50,115 L3,97 Z" fill="#F0F0F0" stroke="#E0E0E0" stroke-width="1" mask="url(#fadeMask)"/>
    <circle cx="50" cy="50" r="30" fill="#57c4f8" stroke="#57c4f8" stroke-width="1"/>
    <circle cx="50" cy="50" r="25" fill="#F0F0F0"/>
    <line x1="50" y1="50" x2="50" y2="30" stroke="#57c4f8" stroke-width="3" stroke-linecap="round"/>
    <line x1="50" y1="50" x2="65" y2="50" stroke="#57c4f8" stroke-width="3" stroke-linecap="round"/>
    <text x="100" y="34" font-family="Arial, sans-serif" font-size="14" fill="#333333">A próxima célula pode levar alguns minutos para ser executada. Por favor, seja paciente.</text>
    <text x="100" y="56" font-family="Arial, sans-serif" font-size="14" fill="#333333">Ignore os avisos e erros, junto com a nota sobre reiniciar o kernel no final.</text>
</svg>

In [ ]:
# Descomente se você tiver problemas de compatibilidade e quiser usar a versão específica da biblioteca sagemaker
# %pip install sagemaker==2.219.0
%pip install --upgrade pip sagemaker boto3

In [4]:
%pip install mlflow==2.13.2 sagemaker-mlflow

Note: you may need to restart the kernel to use updated packages.


c:\Users\csilvapr\OneDrive - NTT DATA EMEAL\Escritorio\Prudential\amazon-sagemaker-from-idea-to-production\.venv\Scripts\python.exe: No module named pip


### Importar pacotes

In [ ]:
import time
import os
import json
import boto3
import numpy as np  
import pandas as pd 
import sagemaker
from time import gmtime, strftime, sleep

(sagemaker.__version__,boto3.__version__)

### Definir constantes

In [3]:
# Obtenha algumas variáveis que você precisa para interagir com o serviço do SageMaker
boto_session = boto3.Session()
region = boto_session.region_name
bucket_name = sagemaker.Session().default_bucket()
bucket_prefix = "from-idea-to-prod/xgboost"  
sm_session = sagemaker.Session()
sm_client = boto_session.client("sagemaker")
sm_role = sagemaker.get_execution_role()
dataset_file_local_path = "data/bank-additional/bank-additional-full.csv"

initialized = True

print(sm_role)

arn:aws:iam::398333718380:role/cfnstudiodomain-SageMakerExecutionRole-8XQVuLUFsn6g


In [5]:
# Armazene algumas variáveis para manter o valor entre os notebooks
%store bucket_name
%store bucket_prefix
%store sm_role
%store region
%store initialized
%store dataset_file_local_path

Stored 'bucket_name' (str)
Stored 'bucket_prefix' (str)
Stored 'sm_role' (str)
Stored 'region' (str)
Stored 'initialized' (bool)
Stored 'dataset_file_local_path' (str)


### Obter ID do domínio
Você precisa desse valor `domain_id` em muitas chamadas do SageMaker Python SDK e do boto3 SageMaker API. O arquivo de metadados do notebook contém o valor `domain_id`. O código a seguir demonstra como acessar o arquivo de metadados do notebook e obter o `domain_id`.

In [ ]:
NOTEBOOK_METADATA_FILE = "/opt/ml/metadata/resource-metadata.json"
domain_id = None

if os.path.exists(NOTEBOOK_METADATA_FILE):
    with open(NOTEBOOK_METADATA_FILE, "rb") as f:
        metadata = json.loads(f.read())
        domain_id = metadata.get('DomainId')
        space_name = metadata.get('SpaceName')
        print(f"ID do domínio SageMaker: {domain_id}")

if not space_name:
    raise Exception(f"Não foi possível encontrar o nome do espaço atual. Certifique-se de que você está executando este notebook em um JupyterLab no SageMaker Studio")
else:
    print(f"Nome do espaço: {space_name}")
    
r = sm_client.describe_space(DomainId=domain_id, SpaceName=space_name)
user_profile_name = r['OwnershipSettings']['OwnerUserProfileName']

assert(user_profile_name)
print(f"Perfil do usuário: {user_profile_name}")

%store domain_id
%store space_name
%store user_profile_name

### Conecte-se ao servidor de rastreamento do MLflow
Se você não tiver um servidor MLflow, siga o procedimento [Developer Guide](https://docs.aws.amazon.com/sagemaker/latest/dg/mlflow-create-tracking-server.html) ou execute a célula de código a seguir para criar uma nova.

Para criar e gerenciar um servidor de rastreamento do MLflow e trabalhar com experiências gerenciadas do MLflow, você precisa das seguintes permissões anexadas à função de execução do SageMaker:

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "sagemaker-mlflow:*",
                "sagemaker:CreateMlflowTrackingServer",
                "sagemaker:UpdateMlflowTrackingServer",
                "sagemaker:DeleteMlflowTrackingServer",
                "sagemaker:StartMlflowTrackingServer",
                "sagemaker:StopMlflowTrackingServer",
                "sagemaker:CreatePresignedMlflowTrackingServerUrl"
            ],
            "Resource": "*"
        }
    ]
}
```

Execute o código a seguir para verificar se você tem um servidor MLflow em execução.

In [7]:
# Localize um servidor MLflow ativo na conta
r = boto3.client("sagemaker").list_mlflow_tracking_servers(
    TrackingServerStatus='Created',
)['TrackingServerSummaries']

if len(r) < 1:
    print("Você não tem nenhum servidor MLflow em execução. Tentando encontrar um servidor no status 'Creating'...")

    r = boto3.client("sagemaker").list_mlflow_tracking_servers(
        TrackingServerStatus='Creating',
    )['TrackingServerSummaries']

    if len(r) < 1:
        print("Você não tem nenhum servidor MLflow no status 'Creating'. Execute a próxima célula de código para criar um novo.")
        mlflow_arn = None
        mlflow_name = None
    else:
        mlflow_arn = r[0]['TrackingServerArn']
        mlflow_name = r[0]['TrackingServerName']
        print(f"Você tem um servidor MLflow {mlflow_arn} no status 'Creating', vamos usar este")
else:
    mlflow_arn = r[0]['TrackingServerArn']
    mlflow_name = r[0]['TrackingServerName']
    print(f"Você tem {len(r)} servidor(es) MLflow em execução. Obtendo o ARN do primeiro servidor: {mlflow_arn}")

You have 1 running MLflow server(s). Get the first server ARN:arn:aws:sagemaker:us-east-1:398333718380:mlflow-tracking-server/mlflow-d-wxmppmljqkj5


In [8]:
# Essa célula de código cria um novo servidor MLflow
if not mlflow_arn:
    ts = strftime('%d-%H-%M-%S', gmtime())
    mlflow_name = f"mlflow-{domain_id}-{ts}"
    r = boto3.client("sagemaker").create_mlflow_tracking_server(
        TrackingServerName=mlflow_name,
        ArtifactStoreUri=f"s3://{bucket_name}/mlflow/{ts}",
        RoleArn=sm_role,
        AutomaticModelRegistration=True,
    )

    mlflow_arn = r['TrackingServerArn']
    print(f"Solicitação de criação do servidor bem-sucedida. O servidor {mlflow_arn} está sendo criado.")

<div style="border: 4px solid coral; text-align: center; margin: auto;">
A criação de um servidor MLflow pode levar até 25 minutos. Você não precisa esperar - prossiga com o fluxo desse treinamento.
</div>

In [9]:
(mlflow_arn, mlflow_name)

('arn:aws:sagemaker:us-east-1:398333718380:mlflow-tracking-server/mlflow-d-wxmppmljqkj5',
 'mlflow-d-wxmppmljqkj5')

In [10]:
%store mlflow_arn
%store mlflow_name

Stored 'mlflow_arn' (str)
Stored 'mlflow_name' (str)


## Instale o Docker para ativar o modo local do Studio
Os aplicativos do Amazon SageMaker Studio suportam o uso do modo local para criar estimadores, processadores e pipelines e, em seguida, implantá-los em um ambiente local. Com o modo local, você pode testar scripts de aprendizado de máquina antes de executá-los em ambientes de treinamento ou hospedagem gerenciados pelo Amazon SageMaker. Consulte [Local mode support in Amazon SageMaker Studio](https://docs.aws.amazon.com/sagemaker/latest/dg/studio-updated-local.html) para entender quais operações do docker o Studio suporta atualmente.

Para usar o modo local nos aplicativos do Studio, você deve instalar o Docker em seu espaço do JupyterLab. 

### Verifique se o acesso ao docker está habilitado


In [ ]:
# verificar se o docker está habilitado no domínio SageMaker
docker_settings = sm_client.describe_domain(DomainId=domain_id)['DomainSettings'].get('DockerSettings')
docker_enabled = False

if docker_settings:
    if docker_settings.get('EnableDockerAccess') in ['ENABLED']:
        print(f"O acesso ao docker está HABILITADO no domínio {domain_id}")
        docker_enabled = True

if not docker_enabled:
    raise Exception(f"Você deve habilitar o acesso ao docker no domínio para usar o modo local do Studio")

<div style="border: 4px solid coral; text-align: center; margin: auto;">
Se a célula de código anterior gerou uma exceção de que o acesso ao docker não está habilitado, você precisará habilitar o acesso. Veja nas instruções a seguir como fazer isso.
</div>

In [31]:
print(f"Domain id: {domain_id}")

Domain id: d-wxmppmljqkj5


### Habilitar o acesso ao docker para o domínio do SageMaker

<div class="alert alert-info">Você só precisará desta seção se o acesso ao docker não estiver ativado no domínio.
</div>

Você precisa da permissão `sagemaker:UpdateDomain` na função de execução.
Essa função precisa ser criada, no IAM -> Roles -> Criar política em linha -> Json e ajustar conforme abaixo:

{
	"Version": "2012-10-17",

	"Statement": [
		{
			"Sid": "Statement1",
			"Effect": "Allow",
			"Action": "sagemaker:UpdateDomain",
			"Resource": "arn:aws:sagemaker:us-east-1:533267005474:domain/d-j2g52sky4w9s"
			
		}
	]
}

Você pode renomea-la como 'AllowSageMakerUpdateDomain'

Não é possível atualizar o domínio a partir desse notebook porque a função de execução do notebook não tem essa permissão. Para atualizar as configurações do domínio, você pode usar uma das seguintes opções.

#### Opção 1: execute `update_domain` no notebook
Se você tiver as permissões correspondentes na função de execução do notebook, poderá executar o seguinte código em um notebook:

```python
import boto3

r = boto3.client('sagemaker').update_domain(
    DomainId=domain_id,
    DomainSettingsForUpdate={
        'DockerSettings': {
            'EnableDockerAccess':'ENABLED',
        }
    }
)
```

#### Opção 2: executar a CLI `aws sagemaker` no terminal
Certifique-se de executar o `AWS CLI` no terminal em que você tem as permissões correspondentes `sagemaker:UpdateDomain`. Execute o seguinte comando:

```
aws sagemaker update-domain --domain-id <DOMAIN-ID> --domain-settings-for-update DockerSettings={EnableDockerAccess='ENABLED'}
```

Por exemplo, você pode executar o comando acima no diretório [AWS CloudShell](https://aws.amazon.com/blogs/aws/aws-cloudshell-command-line-access-to-aws-resources/) em sua conta do AWS.

In [36]:
# verificar as configurações atualizadas
sm_client.describe_domain(DomainId=domain_id)['DomainSettings']

{'DockerSettings': {'EnableDockerAccess': 'ENABLED',
  'VpcOnlyTrustedAccounts': []}}

### Instalar o Docker

In [ ]:
%%bash

# see https://docs.docker.com/engine/install/ubuntu/#install-using-the-repository
sudo apt-get update
sudo apt-get install -y ca-certificates curl
sudo install -m 0755 -d /etc/apt/keyrings
sudo curl -fsSL https://download.docker.com/linux/ubuntu/gpg -o /etc/apt/keyrings/docker.asc
sudo chmod a+r /etc/apt/keyrings/docker.asc

# Adicione o repositório às fontes do Apt:
echo \
  "deb [arch=$(dpkg --print-architecture) signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/ubuntu \
  $(. /etc/os-release && echo "$VERSION_CODENAME") stable" | \
  sudo tee /etc/apt/sources.list.d/docker.list > /dev/null
sudo apt-get update

## Atualmente, apenas a versão 20.10.X do Docker é compatível com o Studio: see https://docs.aws.amazon.com/sagemaker/latest/dg/studio-updated-local.html
# pick the latest patch from:
# apt-cache madison docker-ce | awk '{ print $3 }' | grep -i 20.10
VERSION_STRING=5:20.10.24~3-0~ubuntu-jammy
sudo apt-get install docker-ce-cli=$VERSION_STRING docker-compose-plugin -y

# validar se o Docker Client é capaz de acessar o Docker Server em [unix:///docker/proxy.sock]
docker version

## Dados

Este exemplo usa o [direct marketing dataset](https://archive.ics.uci.edu/ml/datasets/bank+marketing) do ML Repository da UCI:
> [Moro et al., 2014] S. Moro, P. Cortez e P. Rita. Uma Abordagem Baseada em Dados para Prever o Sucesso do Telemarketing Bancário. Sistemas de Suporte à Decisão, Elsevier, 62:22-31, Junho de 2014

Os dados estão relacionados com campanhas de marketing direto de uma instituição bancária portuguesa. As campanhas de marketing eram baseadas em chamadas telefônicas. Muitas vezes, era necessário mais de um contato com o mesmo cliente para verificar se o produto (depósito a prazo bancário) seria ('sim') ou não ('não') subscrito.

Faça o download e descompacte o conjunto de dados:

In [ ]:
!wget -P data/ -N https://archive.ics.uci.edu/static/public/222/bank+marketing.zip --no-check-certificate

In [14]:
import zipfile

with zipfile.ZipFile("data/bank+marketing.zip", "r") as z:
    print("Unzipping bank+marketing...")
    z.extractall("data")

with zipfile.ZipFile("data/bank-additional.zip", "r") as z:
    print("Unzipping bank-additional...")
    z.extractall("data")

print("Done")

Unzipping bank+marketing...
Unzipping bank-additional...
Done


### Veja os dados

In [15]:
df_data = pd.read_csv(dataset_file_local_path, sep=";")

pd.set_option("display.max_columns", 500)  # Vizualize todas as colunas
df_data  # Mostre as cinco primeiras e cinco últimas lunhas do dataframe

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,334,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,383,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,189,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,442,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


### Fazer upload de dados para o S3

In [16]:
input_s3_url = sagemaker.Session().upload_data(
    path=dataset_file_local_path,
    bucket=bucket_name,
    key_prefix=f"{bucket_prefix}/input"
)
print(f"Upload the dataset to {input_s3_url}")

%store input_s3_url

Upload the dataset to s3://sagemaker-us-east-1-398333718380/from-idea-to-prod/xgboost/input/bank-additional-full.csv
Stored 'input_s3_url' (str)


## Reinicie o kernel

In [ ]:
# Reinicie o Kernel para obter os pacotes. 
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

## Fluxo adicional desse treinamento
Você pode continuar o treinamento percorrendo cada um dos seguintes notebooks na ordem direta, por exemplo, 1-2-3-4....

Se estiver interessado em um tópico específico, você pode executar alguns notebooks de forma autônoma, conforme mostrado no fluxograma a seguir:

![](img/workshop-flow.png)

Inicie na etapa 1 [Idea Development](01-idea-development.ipynb) ou etapa 3 [SageMaker Pipelines](03-sagemaker-pipeline.ipynb).

## Recursos adicionais

### Documentação
- [Use Amazon SageMaker Built-in Algorithms](https://docs.aws.amazon.com/sagemaker/latest/dg/algos.html)

### Exemplos práticos
- [Get started with Amazon SageMaker](https://aws.amazon.com/sagemaker/getting-started/)


### Workshops
- [Amazon SageMaker 101 Workshop](https://catalog.us-east-1.prod.workshops.aws/workshops/0c6b8a23-b837-4e0f-b2e2-4a3ffd7d645b/en-US)